### 질문 <-> FewShotChatMessagePrompt 예제

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


examples = [
    {
        "question": "재생",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        넷플릭스, 라디오, 미디어 통합검색, 실시간채널 검색/재생, 음악 검색/재생
        """,
    },
    {
        "question": "경로",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        내비정보, 대중교통, 버스, 택시, 지하철
        """,
    },
    {
        "question": "생활 정보",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        날씨 정보 요청, 뉴스 브리핑, 뉴스속보 요청, 환율 정보 요청, 프로야구
        """,
    },
    {
        "question": "채팅",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        가이드 발화, 감성채팅, 미분류, 미응대, 서비스 제안
        """,
    },
    {
        "question": "오늘 일정 알려줘",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        일정목록 입력, 일정목록 조회
        """,
    },
    {
        "question": "메모 팁 알려줘",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        리마인더, 음성 메모, 음성 메시지
        """,
    },
    {
        "question": "법률 정보 궁금해",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        법제처, 기본법령, 국가법령정보센터, 자치법규정보시스템, 한국법령정보원, 헌법재판소, 정부24, 입법 동향
        """,
    },
]


to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
  vectorstore=vectorstore,
  k=1 # 생성할 예제 개수입니다.
)


example_intent_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_intent_prompt,
#     examples=examples,
# )


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_intent_prompt,
)


intent_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 질문에 대한 의도를 정확하게 파악하여 관련성이 있는 단어를 리스트로 생성할 수 있는 시스템입니다.
     다음의 세가지 내용을 참고하여 입력 받은 질문에 대해 처리해주세요.
     - 입력 받은 질문의 의도를 파악한 결과가 예시로 학습한 정보와 유사성을 띄고 있으면 가장 유사도가 높은 예시의 답변을 그대로 반환해주세요.
     - 예시와 유사성을 띄지 않는 다면 해당 질문의 의도와 관련성이 있는 단어를 추가적으로 5개 생해주세요.
     - 위의 과정을 모두 마쳤으면 추가적인 답변은 필요없습니다."""),
     few_shot_prompt,
    ("human", "{question}")
])


# intent_prompt = ChatPromptTemplate.from_messages([
#     ("system", """당신은 질문에 대한 의도를 정확하게 파악할 수 있는 전문가입니다.
#      질문의 맥락을 파악하고 그 의도를 대표할 수 있는 한 개의 단어를 생각해보세요. 그리고 그 한 개의 단어를 기반으로 아래의 내용을 처리해주세요.
#      - 질문의 의도를 대표하는 단어가 예시로 학습한 정보와 유사성을 띄고 있으면 가장 유사도가 높은 예시의 답변을 그대로 반환해주세요.
#      - 질문의 의도를 대표하는 단어가 예시와 유사성을 띄지 않는 다면 자체적으로 관련성이 있는 단어를 추론하여 추가적으로 5개를 생해주세요.
#      - 위의 과정을 모두 마쳤으면 추가적인 답변은 필요없습니다.
     
#      질
#      """),
#      few_shot_prompt,
#     ("human", "{question}")
# ])



intent_chain = intent_prompt | chat


# intent_chain.invoke({
#     "question": "영화 타이타닉 틀어줘."
# })


# intent_chain.invoke({
#     "question": "스프링클러 설치시 참고해야하는 관련 법률 정보를 알려줘."
# })





# result = intent_chain.invoke({
#     "question": "서울역에서 삼성역 가는 방법 알려줘."
# })



result = intent_chain.invoke({
    "question": "법률 정보가 궁금해"
})


# result = intent_chain.invoke({
#     "question": "스프링클러 설치시 참고해야하는 관련 법률 정보를 알려줘."
# })

# result = intent_chain.invoke({
#     "question": "오늘 한국의 메인 뉴스 기사가 뭐야?"
# })

# result = intent_chain.invoke({
#     "question": "영화 타이타닉 틀어줘."
# })


result.content

입력하신 질문의 의도를 파악한 결과, "법률 정보"에 대한 정보를 얻고자 하는 것으로 이해하였습니다. 아래는 관련성이 있는 단어 5개입니다:
1. 법률
2. 규정
3. 법령
4. 법제처
5. 법률정보원

'입력하신 질문의 의도를 파악한 결과, "법률 정보"에 대한 정보를 얻고자 하는 것으로 이해하였습니다. 아래는 관련성이 있는 단어 5개입니다:\n1. 법률\n2. 규정\n3. 법령\n4. 법제처\n5. 법률정보원'